In [ ]:
from pathlib import Path
import logging
from dotenv import dotenv_values

from generator_drainage_units.generator_order_levels import GeneratorOrderLevels

In [ ]:
logging.basicConfig(level=logging.DEBUG)

config = dotenv_values("..\\.env")
base_dir = config["BASE_DIR"]
# case_name = "vallei_en_veluwe"
# case_name = "geerestein"
case_name = "hattemerbroek"
# case_name = "pangelerbeek"

In [ ]:
def run_generator_order_levels(
    path: Path,
    read_results: bool = True,
    write_results: bool = True
) -> GeneratorOrderLevels:
    
    order_generator = GeneratorOrderLevels(
        read_results=read_results, write_results=write_results
    )
    order_generator.read_data_from_case(path=path)
    
    return order_generator

In [ ]:
case_path = Path(base_dir, case_name)

order_generator = run_generator_order_levels(path=case_path)

In [ ]:
def get_len_gdf(gdf):
    return len(gdf)

get_len_gdf(order_generator.hydroobjecten)